In [1]:
import itertools
import numpy as np
import os
from tqdm import tqdm
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
import functools

%load_ext autoreload
%autoreload 2

from genpen.utils import Paper
import bezier
from dataclasses import asdict, dataclass, field

In [2]:
# make page
paper_size = '11x17 inches'
border:float=35
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

In [206]:
theta = np.linspace(0, np.pi * 70, 5000)
x = np.cos(theta) ** 4 * np.sin(theta) - (theta * 0.002)
y = theta * 0.005 * np.sin(0.99 * theta) + (theta * 0.002)
ls = LineString(np.stack([x,y]).T * 150)


In [207]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.geometry(ls)

sk.display(color_mode='none')

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-21T19:07:41.049519 
 
 
 
 
 
 <polyline points="652.6742,566.8689 677.4598,566.9242 701.6235,566.9903 724.5658,567.0673 745.7304,567.1549 764.6241,567.253 780.8331,567.3615 794.0369,567.4801 804.0174,567.6085 810.6648,567.7465 813.9778,567.8938 814.0608,568.0498 811.116,568.2144 805.4317,568.3869 797.3685,568.5669 787.3416,568.7539 775.8025,568.9474 763.2193,569.1468 750.0577,569.3515 736.7623,569.5609 723.7403,569.7744 711.347,569.9912 699.8756,570.2106 689.5486,570.4321 680.5154,570.6549 672.851,570.8782 666.5608,571.1012 661.5871,571.3234 657.8194,571.5438 655.1063,571.7617 653.2697,571.9764 652.1193,572.1871 651.4668,572.393 651.1404,572.5933 650.9962,572.7874 650.929,572.9744 650.8786,573.1537 650.8346,573.3245 650.8368,573.4861 650.9725,573.6379 651.3706,573.7791 652.1928,573.9091 653.6221,574.0274 655.8496,574.1333 659.0605,574.2262 663.4187,574.3057 669.0535,574.3712 676.0456,574.4223 684.4171,574.4585 694.1221,574.4795 705.0425,574.4849 716.9858,574.4744 729.6873,574.4477 742.8164,574.4046 755.9858,574.3449 768.7646,574.2685 780.6936,574.1754 791.3034,574.0653 800.1333,573.9385 806.7506,573.7948 810.7701,573.6345 811.8714,573.4576 809.8152,573.2644 804.4555,573.0552 795.749,572.8301 783.7605,572.5896 768.6636,572.3341 750.7372,572.064 730.358,571.7797 707.9881,571.4819 684.1599,571.1712 659.4576,570.8481 634.4963,570.5133 609.9006,570.1675 586.2821,569.8116 564.2174,569.4462 544.2281,569.0723 526.7626,568.6907 512.1816,568.3023 500.7464,567.908 492.612,567.5089 487.8246,567.1059 486.3228,566.7001 487.9439,566.2924 492.4335,565.8841 499.459,565.4761 508.6258,565.0696 519.495,564.6657 531.6029,564.2655 544.4807,563.8703 557.6727,563.4811 570.7546,563.0991 583.3479,562.7254 595.1327,562.3613 605.8564,562.0078 615.3391,561.6662 623.4754,561.3375 630.2316,561.0229 635.6402,560.7234 639.7915,560.4402 642.8213,560.1744 644.8985,559.9268 646.2102,559.6987 646.9471,559.4909 647.2896,559.3044 647.3943,559.1401 647.3836,558.9988 647.3371,558.8815 647.286,558.7889 647.2111,558.7217 647.0444,558.6807 646.6732,558.6665 645.9482,558.6796 644.6937,558.7206 642.7202,558.79 639.8385,558.8883 635.8738,559.0157 630.681,559.1726 624.1576,559.3592 616.2563,559.5758 606.9939,559.8223 596.4582,560.0989 584.8108,560.4055 572.2871,560.742 559.1914,561.1084 545.8888,561.5042 532.794,561.9293 520.3564,562.3833 509.0435,562.8658 499.3219,563.3762 491.6383,563.914 486.3999,564.4785 483.9558,565.069 484.5808,565.6847 488.4609,566.3249 495.6827,566.9885 506.2266,567.6747 519.964,568.3824 536.6587,569.1105 555.9735,569.8578 577.4798,570.6232 600.6725,571.4054 624.9863,572.2031 649.8163,573.0149 674.5391,573.8394 698.5351,574.6752 721.2106,575.5207 742.019,576.3745 760.4793,577.2349 776.1932,578.1004 788.8569,578.9693 798.2708,579.84 804.3437,580.7107 807.0928,581.5798 806.6398,582.4456 803.2027,583.3063 797.0833,584.1602 788.6527,585.0056 778.3337,585.8406 766.5822,586.6635 753.8672,587.4727 740.6519,588.2663 727.3757,589.0427 714.4378,589.8002 702.1837,590.537 690.8946,591.2515 680.781,591.9421 671.979,592.6073 664.5517,593.2453 658.4929,593.8548 653.7351,594.4343 650.1598,594.9823 647.6102,595.4974 645.9049,595.9784 644.8528,596.424 644.2678,596.8329 643.9819,597.2041 643.8573,597.5364 643.7958,597.8289 643.7459,598.0806 643.7056,598.2906 643.7232,598.4583 643.8937,598.5829 644.3523,598.6638 645.2652,598.7004 646.8177,598.6923 649.2011,598.6392 652.5978,598.5408 657.1669,598.397 663.0301,598.2075 670.2589,597.9725 678.8642,597.6921 688.788,597.3664 699.8995,596.9957 711.9934,596.5805 724.7935,596.1212 737.9584,595.6184 751.0924,595.0728 763.7586,594.4851 775.4948,593.8563 785.8317,593.1872 794.312,592.4789 800.5097,591.7325 804.0496,590.9493 804.6243,590.1306 802.0094,589.2778 796.076,588.3923 786.7992,587.4757 774.2625,586.5296 758.6579,585.5558 740.2818,584.5561 719.5264,583.5322 696.867,582.4861 672.8463,581.4198 648.0553,580.3353 623.1124,579.2348 598.6416,578.1202 575.

In [208]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/237_no_loops.svg')

In [209]:
# make page
paper_size = '11x17 inches'
border:float=35
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

In [254]:
theta = np.linspace(0, np.pi * 450, 40000)
x = np.cos(theta) ** 7 * np.sin(theta) - (theta * 0.004)
y = theta * 0.0045 * np.sin(0.99 * theta) + (theta * 0.002)
ls = LineString(np.stack([x,y]).T * 25)


In [255]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.geometry(ls)

sk.display(color_mode='none')

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-21T19:50:48.655327 
 
 
 
 
 
 <polyline points="795.1591,549.8263 798.47,549.8335 801.6901,549.8417 804.7324,549.851 807.517,549.8614 809.9749,549.8728 812.0499,549.8852 813.7011,549.8986 814.904,549.9129 815.6506,549.9283 815.9493,549.9446 815.8235,549.9618 815.3098,549.98 814.4559,549.999 813.3174,550.0188 811.9552,550.0394 810.4322,550.0608 808.8105,550.083 807.1489,550.1058 805.5008,550.1293 803.9118,550.1535 802.4195,550.1781 801.0519,550.2033 799.8281,550.229 798.7584,550.2551 797.845,550.2816 797.0835,550.3084 796.4638,550.3354 795.9722,550.3627 795.5922,550.3901 795.3063,550.4177 795.0972,550.4452 794.9484,550.4728 794.8452,550.5003 794.775,550.5277 794.7277,550.5548 794.6953,550.5818 794.6721,550.6084 794.6542,550.6346 794.639,550.6605 794.6249,550.6858 794.6114,550.7106 794.598,550.7348 794.5847,550.7583 794.5713,550.7811 794.5579,550.8031 794.5446,550.8243 794.5312,550.8446 794.5178,550.864 794.5042,550.8823 794.4901,550.8997 794.4747,550.9159 794.4563,550.931 794.4323,550.9449 794.3986,550.9576 794.3491,550.969 794.2758,550.9791 794.168,550.9879 794.0131,550.9952 793.7962,551.0012 793.5005,551.0056 793.1088,551.0086 792.6034,551.0101 791.9683,551.0101 791.1899,551.0085 790.259,551.0053 789.1715,551.0005 787.9308,550.9941 786.548,550.9861 785.0431,550.9765 783.4454,550.9653 781.7932,550.9524 780.1334,550.9379 778.5199,550.9218 777.0119,550.904 775.6719,550.8846 774.5627,550.8637 773.7447,550.8411 773.2728,550.817 773.1934,550.7914 773.5419,550.7642 774.3401,550.7356 775.5945,550.7055 777.2954,550.674 779.4161,550.6411 781.9141,550.6069 784.7316,550.5713 787.7982,550.5346 791.0331,550.4966 794.3486,550.4574 797.6538,550.4172 800.858,550.3759 803.8749,550.3337 806.6258,550.2905 809.0428,550.2465 811.0715,550.2017 812.6729,550.1562 813.8242,550.11 814.5193,550.0633 814.7684,550.0161 814.5966,549.9684 814.0418,549.9204 813.1527,549.8721 811.9857,549.8237 810.6021,549.7752 809.0648,549.7266 807.4357,549.6782 805.7732,549.6299 804.1296,549.5818 802.5499,549.5341 801.0704,549.4868 799.7183,549.44 798.5117,549.3938 797.4597,549.3484 796.5639,549.3037 795.819,549.2598 795.2146,549.217 794.7365,549.1752 794.368,549.1346 794.0917,549.0951 793.8902,549.057 793.7473,549.0203 793.6484,548.9851 793.5813,548.9515 793.536,548.9196 793.5049,548.8893 793.4825,548.8609 793.465,548.8344 793.4499,548.8099 793.436,548.7874 793.4225,548.767 793.4091,548.7488 793.3958,548.7329 793.3824,548.7193 793.369,548.7081 793.3557,548.6993 793.3423,548.693 793.3289,548.6893 793.3153,548.6881 793.3011,548.6896 793.2854,548.6938 793.2666,548.7007 793.2417,548.7104 793.2065,548.7229 793.1548,548.7382 793.0781,548.7563 792.9657,548.7773 792.8046,548.8012 792.5796,548.828 792.2739,548.8577 791.8701,548.8903 791.3509,548.9258 790.7001,548.9642 789.9049,549.0055 788.9562,549.0497 787.8511,549.0968 786.5936,549.1467 785.1957,549.1994 783.6786,549.2549 782.0727,549.3132 780.4172,549.3742 778.7598,549.4379 777.1552,549.5042 775.663,549.5731 774.346,549.6445 773.2669,549.7183 772.4855,549.7946 772.0559,549.8732 772.0235,549.9541 772.4222,550.0371 773.272,550.1222 774.5778,550.2094 776.3279,550.2985 778.4938,550.3894 781.0312,550.4821 783.8807,550.5765 786.9706,550.6724 790.2191,550.7697 793.5381,550.8684 796.8363,550.9683 800.0235,551.0693 803.0141,551.1714 805.7303,551.2743 808.1057,551.3779 810.0877,551.4823 811.6389,551.5871 812.7385,551.6923 813.3822,551.7978 813.5821,551.9034 813.3647,552.009 812.7694,552.1145 811.8459,552.2197 810.6513,552.3245 809.2471,552.4288 807.6963,552.5324 806.0606,552.6352 804.3977,552.7371 802.7592,552.8379 801.1892,552.9374 799.723,553.0357 798.3867,553.1324 797.1972,553.2275 796.163,553.3209 795.2847,553.4124 794.5563,553.5018 793.967,553.5892 793.5021,553.6742 793.145,553.7569 792.8781,553.8371 792.684,553.9146 792.5467,553.9893 792.452,554.0612 792.3878,554.1301 792.3445,554.196 792.3147,554.2586 792.293,554.3179 792.2758,554.3738 792.261,554.4262 792.

In [256]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/239_no_loops.svg')

In [257]:
# make page
paper_size = '11x17 inches'
border:float=35
paper = Paper(paper_size)
drawbox = paper.get_drawbox(border)

In [334]:
lss = []

theta = np.linspace(0, np.pi * 550, 40000)
y = np.cos(theta* 0.005) * np.cos(theta) ** 7 * np.sin(theta) - (theta * 0.018)
x = theta * 0.0025 * np.sin(0.99 * theta) + (theta * 0.004) 
ls = gp.make_like(LineString(np.stack([x,y]).T), drawbox)
lss.append(ls)

theta = np.linspace(0, np.pi * 550, 40000)
y = np.cos(theta* 0.005) * np.cos(theta) ** 7 * np.sin(theta) - (theta * 0.018)
x = -1 * (theta * 0.0025 * np.sin(0.99 * theta) + (theta * 0.004) )
ls = gp.make_like(LineString(np.stack([x,y]).T), drawbox)
lss.append(ls)

theta = np.linspace(0, np.pi * 550, 40000)
y = np.cos(theta* 0.003) * np.cos(theta) ** 7 * np.sin(theta) - (theta * 0.015)
x = theta * 0.0025 * np.sin(0.99 * theta)
ls = gp.make_like(LineString(np.stack([x,y]).T), drawbox)
lss.append(ls)

In [335]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(lss):
    sk.stroke(i + 1)
    sk.geometry(layer)
sk.display(color_mode='none')

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-22T01:01:45.283064 
 
 
 
 
 
 <polyline points="283.6114,1490.3647 283.6191,1492.198 283.6273,1493.9553 283.6358,1495.5653 283.6447,1496.9658 283.654,1498.1071 283.6637,1498.9549 283.6738,1499.4915 283.6842,1499.7165 283.695,1499.6454 283.7061,1499.3078 283.7176,1498.7444 283.7294,1498.0036 283.7414,1497.1377 283.7538,1496.1992 283.7664,1495.2373 283.7793,1494.2956 283.7924,1493.4094 283.8057,1492.605 283.8191,1491.8994 283.8328,1491.3003 283.8465,1490.8077 283.8604,1490.4153 283.8744,1490.112 283.8884,1489.8842 283.9024,1489.7172 283.9165,1489.5965 283.9305,1489.5095 283.9445,1489.4452 283.9584,1489.3954 283.9722,1489.354 283.9859,1489.3171 283.9994,1489.2822 284.0127,1489.248 284.0258,1489.2141 284.0387,1489.1803 284.0513,1489.1465 284.0635,1489.1126 284.0755,1489.0788 284.0871,1489.0449 284.0984,1489.011 284.1092,1488.9767 284.1197,1488.9414 284.1296,1488.9035 284.1392,1488.8603 284.1482,1488.8073 284.1568,1488.7377 284.1648,1488.6426 284.1723,1488.5106 284.1792,1488.3283 284.1856,1488.0813 284.1914,1487.7548 284.1966,1487.3358 284.2012,1486.8145 284.2052,1486.1858 284.2085,1485.452 284.2113,1484.6231 284.2134,1483.7187 284.215,1482.7677 284.2158,1481.8077 284.2161,1480.8841 284.2158,1480.0473 284.2148,1479.3502 284.2133,1478.8443 284.2111,1478.5763 284.2083,1478.5841 284.205,1478.8938 284.2011,1479.5166 284.1967,1480.4473 284.1918,1481.6638 284.1863,1483.1272 284.1804,1484.7841 284.174,1486.5695 284.1671,1488.4102 284.1599,1490.2299 284.1522,1491.9533 284.1442,1493.5113 284.1359,1494.8448 284.1272,1495.908 284.1183,1496.6712 284.1092,1497.1216 284.0999,1497.2631 284.0904,1497.1153 284.0808,1496.7113 284.0711,1496.0942 284.0614,1495.3138 284.0516,1494.423 284.0419,1493.4735 284.0322,1492.5133 284.0227,1491.5838 284.0133,1490.7179 284.0042,1489.9395 283.9952,1489.2628 283.9866,1488.6933 283.9782,1488.2292 283.9703,1487.8625 283.9627,1487.5815 283.9556,1487.3719 283.949,1487.219 283.9429,1487.1088 283.9373,1487.029 283.9324,1486.9694 283.9282,1486.9225 283.9246,1486.8826 283.9217,1486.8464 283.9196,1486.8118 283.9183,1486.7777 283.9178,1486.7439 283.9182,1486.71 283.9195,1486.6762 283.9216,1486.6424 283.9247,1486.6085 283.9288,1486.5747 283.9339,1486.5407 283.94,1486.5062 283.9471,1486.4704 283.9553,1486.4314 283.9645,1486.386 283.9748,1486.3292 283.9862,1486.2536 283.9987,1486.1496 284.0123,1486.0053 284.027,1485.8067 284.0429,1485.5394 284.0598,1485.1888 284.0779,1484.7427 284.097,1484.1922 284.1173,1483.5344 284.1386,1482.7733 284.161,1481.9216 284.1845,1481.0014 284.209,1480.0441 284.2345,1479.0897 284.261,1478.1851 284.2885,1477.3816 284.317,1476.7323 284.3463,1476.2875 284.3766,1476.0919 284.4077,1476.1806 284.4396,1476.5757 284.4723,1477.284 284.5057,1478.2958 284.5398,1479.5842 284.5746,1481.106 284.61,1482.8045 284.6459,1484.6119 284.6823,1486.4538 284.7192,1488.2536 284.7565,1489.9375 284.7941,1491.4386 284.832,1492.7013 284.8701,1493.684 284.9084,1494.3616 284.9468,1494.7259 284.9853,1494.7853 285.0237,1494.5634 285.0621,1494.0964 285.1004,1493.4296 285.1384,1492.6139 285.1762,1491.7022 285.2136,1490.7456 285.2507,1489.7904 285.2873,1488.8759 285.3234,1488.0324 285.3589,1487.2812 285.3938,1486.6339 285.428,1486.0941 285.4614,1485.6579 285.494,1485.3162 285.5257,1485.0563 285.5565,1484.8638 285.5862,1484.724 285.6149,1484.6233 285.6425,1484.55 285.669,1484.4946 285.6942,1484.45 285.7182,1484.4114 285.7409,1484.3758 285.7622,1484.3414 285.7822,1484.3075 285.8007,1484.2736 285.8177,1484.2398 285.8333,1484.2059 285.8473,1484.1721 285.8598,1484.1382 285.8706,1484.1044 285.8799,1484.0703 285.8875,1484.0356 285.8935,1483.9991 285.8978,1483.9588 285.9004,1483.9108 285.9014,1483.8497 285.9007,1483.7674 285.8983,1483.6537 285.8942,1483.4959 285.8884,1483.2801 285.881,1482.9914 285.8719,1482.6159 285.8611,1482.142 285.8488,1481.5624 285.8348,1480.8759 285.8193,1480.0887 285.8022,1479.2161 285.7836,1478.2827 285.7635,1477.3224 285.742,1476.3773 285.719

In [336]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/240_no_loops.svg')